
Optionals: 
1. We might need to implement random data at the end of the project.

# Import of Important Library, Network and Meta Data

In [1]:
import igraph as ig
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Import Network data
networkData = pd.read_csv('TD3_primary_undirected_temporal.csv')
networkData

,time,node-i,node-j
0,0,58,63
1,0,59,64
2,0,63,66
3,0,85,190
4,0,85,214
...,...,...,...
125768,3099,152,172
125769,3099,152,174
125770,3099,167,180
125771,3099,172,178


In [3]:
# Import MetaData [information on the nodes (class, gender)]
metadata = pd.read_csv("TD3_primary_metadata.csv")
metadata

,node-index,class,gender
0,0,5B,M
1,1,5B,F
2,2,5B,M
3,3,5B,F
4,4,5B,M
...,...,...,...
237,237,2A,M
238,238,2A,F
239,239,2A,M
240,240,1B,F


# Initialisation of Simulation

In [8]:
numOfNodes = metadata.shape[0]
print("Number of Nodes: ", numOfNodes)
susceptiblePopuilation = [idx for idx in range(numOfNodes)]
exposedPopulation, infectiousPopulation, recoveredPopulation = [], [], []

Number of Nodes:  242


## Initialisation of Patient Zero

In [ ]:
patientZero = np.random.choice(susceptiblePopuilation)
susceptiblePopuilation.remove(patientZero)
exposedPopulation.append(patientZero)